In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t
import nglview as nv
from simtk import unit
import numpy as np

/home/diego/Myopt/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1170: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


_ColormakerRegistry()

# Cálculo de distancias

El cálculo de distancias en MolModMT se puede hacer actualmente recurriendo a dos motores: el propio conjunto de librerías de MolMoMT programadas para tal efecto y las encontradas en la librería MDTraj (en el futuro se añadirán MDAnalysis y Pytraj). La opción por defecto, si no se especifica otra cosa, es usar el código propio de MolModMT. Si por algún motivo se prefiere realizar los cálculos empleando métodos de MDTraj, se puede añadir el argumento de entrada `engine='MDTraj'` a cada uno de los métodos que aparecen a continuación.

## Objeto XYZ

MolModMT acepta trabajar con matrices de coordenadas espaciales como la forma más simple de codificar las posiciones de un modelo molecular de átomos iguales, sin información topológica. Es lo que MolModMT denomina objeto `XYZ`:

In [3]:
model_A = np.zeros([6,3], dtype='float64') * unit.angstroms

In [4]:
m3t.get_form(model_A)

'XYZ'

In [5]:
m3t.get(model_A, n_frames=True, n_atoms=True)

[1, 6]

El objeto XYZ debe ser un tensor de tipo array de numpy de rango 3 o 2 según sea: [índice_frame, indice_átomo, coordenada] o [índice_atómo, coordenada], donde coordenada es x, y o z. Es obligatorio que este array de numpy lleve unidades de longitud.

In [6]:
model_A = np.zeros([2,4,3], dtype='float64') * unit.nanometers

In [7]:
m3t.get(model_A, n_frames=True, n_atoms=True)

[2, 4]

En adelante, y para ilustrar la funcionalidad de los métodos aquí presentados, supongamos dos modelos con coordenadas espaciales para tres frames, el primero con tres elementos o átomos y el segundo con dos:

In [8]:
# Set_A inicializado a cero:
set_A = np.zeros([3,3,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_A para los tres frames 
set_A[0,0,:] = [0, 2, -1] * unit.nanometers
set_A[1,0,:] = [1, 2, -1] * unit.nanometers
set_A[2,0,:] = [0, 2, -1] * unit.nanometers

## coordenadas del segundo elemento de Set_A para los tres frames
set_A[0,1,:] = [-1, 1, 1] * unit.nanometers
set_A[1,1,:] = [-1, 0, 1] * unit.nanometers
set_A[2,1,:] = [0, 0, 1] * unit.nanometers

## coordenadas del tercer elemento de Set_A para los tres frames
set_A[0,2,:] = [-2, 0, 1] * unit.nanometers
set_A[1,2,:] = [-2, 0, 0] * unit.nanometers
set_A[2,2,:] = [-1, 1, 0] * unit.nanometers

# Set_B inicializado a cero:

set_B = np.zeros([3,2,3], dtype='float64') * unit.nanometers

## coordenadas del primer elemento de Set_B para los tres frames
set_B[0,0,:] = [4, -2, 0] * unit.nanometers
set_B[1,0,:] = [5, -2, -1] * unit.nanometers
set_B[2,0,:] = [5, -2, 0] * unit.nanometers

## coordenadas del segundo elemento de Set_B para los tres frames
set_B[0,1,:] = [3, 0, -1] * unit.nanometers
set_B[1,1,:] = [3, 1, 0] * unit.nanometers
set_B[2,1,:] = [4, 1, 1] * unit.nanometers

## Distancias entre elementos

El caso más sencillo que podemos resolver en cuanto al cálculo de distancias es el de la distancia entre los puntos de un conjunto, o entre dos conjuntos distintos de puntos en el espacio, sin condiciones de contorno periódicas. 

Supongamos que queremos calcular las distancias entre todos los elementos de `set_A` para cada uno de sus frames:

In [9]:
distances = m3t.distance(item_1=set_A)

El resultado es un tensor de rango 3 donde el primer rango es el índice de frame, en este caso ha tres, y el segundo y el tercer rango corresponden a los indices de los elementos de `set_A`:

In [10]:
distances.shape

(3, 3, 3)

De tal manera que la distancia entre el elemento 0-ésimo y el 2-ésimo en el frame 0-ésimo es:

In [11]:
print(distances[0,0,2])

3.4641016151377544 nm


Si quisíeramos haber calculado únicamente la distancia entre esos dos elementos podríamos haber recurrido a:

In [12]:
distances = m3t.distance(item_1=set_A, selection_1=0, selection_2=2)

In [13]:
print(distances)

[[[3.46410162]]

 [[3.74165739]]

 [[1.73205081]]] nm


Observa que la forma del array ahora es un tensor de rango 3 donde en este caso el primer rango tiene dimensión 3 (por ser tres frames) y para el resto las dimensiones son iguales a 1:

In [14]:
distances.shape

(3, 1, 1)

Esto es porque tenemos tres frames (primer rango) con dos listas de elementos (segundo y tercer rango): `selection_1` y `selection_2`. Y en este caso ambas listas contienen un único elemento, el 0-ésimo en el primer caso y el 2-ésimo en el segundo. Veamos ahora qué pasa si queremos calcular la distancia entre el elemento 0-ésimo y los otros dos:

In [15]:
distances = m3t.distance(item_1=set_A, selection_1=0, selection_2=[1,2])

In [16]:
distances.shape

(3, 1, 2)

El tensor de distancias tiene ahora dimensión 2 en el último rango. ¿Cómo entonces por ejemplo veríamos cúal es la distancia entre el elemento 0-ésimo y el 2-ésimo para el frame 1-ésimo según el actual objeto `distances`? 

In [17]:
print(distances[1,0,1])

3.7416573867739413 nm


Para mayor claridad podemos también obtener el resultado del método `molmodmt.distance` como diccionario, donde ahora sí podemos recurrir al índice de elemento expresado en los argumentos `selection_1` y `selection_2`:

In [18]:
distances = m3t.distance(item_1=set_A, selection_1=0, selection_2=[1,2], output_form='dict')

In [19]:
print(distances)

{0: {1: {0: Quantity(value=2.449489742783178, unit=nanometer), 1: Quantity(value=3.4641016151377544, unit=nanometer), 2: Quantity(value=2.8284271247461903, unit=nanometer)}, 2: {0: Quantity(value=3.4641016151377544, unit=nanometer), 1: Quantity(value=3.7416573867739413, unit=nanometer), 2: Quantity(value=1.7320508075688772, unit=nanometer)}}}


En realidad el objeto `distances` es ahora un diccionario de diccionarios para poder rápidamente conocer las distancias entre dos elementos en función del índice de frame. Veamos cómo acceder a la lista de distancias para los diferentes frames (aquí sólo uno) entre el elemento 0-ésimo incluido en `selection_1` y el 2-ésimo incluido en `selection_2` para el frame 1-ésimo:

In [20]:
print(distances[0][2][1])

3.7416573867739413 nm


Supongamos ahora que tenemos los dos conjuntos de puntos en el espacio en tres instantes de tiempo: `set_A` y `set_B`.

Podemos ahora invocando una sola vez el método `molmodmt.distance` calcular todas las distancias entre los elementos de ambos conjuntos para todos los frames:

In [21]:
distances = m3t.distance(item_1=set_A, item_2=set_B)

De esta manera podemos por ejemplo acceder a la distancia entre el elemento 2-ésimo de `set_A` y el elemento 0-ésimo de `set_B` para el segundo frame o frame 1-ésimo.

In [22]:
print(distances[1,2,0])

7.3484692283495345 nm


O imprimir esta distancia en función para todos los frames:

In [23]:
print(distances[:,2,0])

[6.40312424 7.34846923 6.70820393] nm


Como vimos en el caso anterior, podemos ordenar que la salida del método `molmodmt.distance` sea en forma de diccionario de diccionarios.

In [24]:
distances = m3t.distance(item_1=set_A, item_2=set_B, output_form='dict')

Podemos entonces acceder a las mismas distancias que anteriormente:

In [25]:
# distancia entre el elemento 2-ésimo de set_A y 0-ésimo para el segundo frame
print(distances[2][0][1])

7.3484692283495345 nm


In [26]:
# distancia entre el elemento 2-ésimo de set_A y 0-ésimo en cada frame
print(distances[2][0])

{0: Quantity(value=6.4031242374328485, unit=nanometer), 1: Quantity(value=7.3484692283495345, unit=nanometer), 2: Quantity(value=6.708203932499369, unit=nanometer)}


Esta manera de sacar los datos puede ser muy conveniente cuando calculamos distancias entre listas de elementos de ambos conjuntos. Supongamos que queremos calcular las distancias entre los elementos 1-ésimo y 2-ésimo de `set_A` y el elemento 1-ésimo de `set_B` para todos los frames. Primero veamos cómo hacerlo con la salida tipo tensor (salida por defecto):

In [27]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], item_2=set_B, selection_2=1)

De esta manera las distancias por ejemplo para ambos frames entre el elemento 2-ésimo de `set_A` con el elemento 1-ésimo de `set_B` se situan ahora en la posición 1-ésima (por ser el 1-ésimo elemento de la lista `selection_1`) frente a la 0-ésima (por ser el elemento 0-ésimo elemento, único en este caso, de la lista `selection_2`) en el segundo y tercer rango del tensor correspondientemente:

In [28]:
print(distances[:,1,0])

[5.38516481 5.09901951 5.09901951] nm


La salida de `molmodmt.distances` tipo diccionario de diccionarios puede hacer más cómodo el trabajo con las distancias resultado, ya que el diccionario recupera los índices iniciales de cada elemento en ambos conjuntos `set_A` y `set_B`.

In [29]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], item_2=set_B, selection_2=1, output_form='dict')

La lista de distancias para los distintos frames entre el elemento 2-ésimo de `set_A` y el 1-ésimo de `set_B`, las mismas que anteriormente, puede ser encontrada según: 

In [30]:
print(distances[2][1])

{0: Quantity(value=5.385164807134504, unit=nanometer), 1: Quantity(value=5.0990195135927845, unit=nanometer), 2: Quantity(value=5.0990195135927845, unit=nanometer)}


Por último, supongamos que únicamente queremos calcular la distancia entre estas dos últimas selecciones para el frame 0-ésimo y el 2-ésimo. Podemos incluir también como argumento de entrada en `molmodmt.distance` la selección de una lista de frames:

In [31]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,2],
                         item_2=set_B, selection_2=1, frame_indices_2=[0,2])

Esta vez el resultado es un tensor donde ahora ya no tiene dimensión 3 en el primer rango, sino 2 (dos frames):

In [32]:
distances.shape

(2, 2, 1)

Supongamos que queremos acceder a la distancia entre el elemento 2-ésimo de `set_A` con el elemento 1-ésimo de `set_B` en el frame 2-ésimo:

In [33]:
print(distances[1,1,0])

5.0990195135927845 nm


Puedes comprobar que dado que `frame_indices_1` y `frame_indices_2` son iguales, puedes prescindir de `frame_indices_2`. El resultado será el mismo ya que el método sobreentiende que se trata de la misma selección de índices de frame cuando `frame_indices_2=None` y calcula las distancias entre las respectivas selecciones de elementos de ambos items en cada uno de los frames indicados únicamente en `frame_indices_1`:

In [34]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,2],
                         item_2=set_B, selection_2=1)

In [35]:
distances.shape

(2, 2, 1)

En esta situación en la que estamos incluyendo una lista de indices de frames, la salida de `molmodmt.distance` en forma de diccionario de diccionarios tambien respeta los índices originales en el caso de los frames:

In [36]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,2],
                         item_2=set_B, selection_2=1, output_form='dict')

Ahora podemos conocer las distancias entre dos elementos en un frame determinado sin tener que acudir a los indices de dichos elementos y frame que tienen en las listas `selection_1`, `selection_2` y `frame_indices_1`. Basta con los índices absolutos originales que para el caso anterior son 2-ésimo de `set_A` y 1-ésimo de `set_B` en el frame 2-ésimo:

In [37]:
print(distances[2][1][2])

5.0990195135927845 nm


## Distancias de desplazamiento o distancias entre distintas selecciones de índices de frame

La final de la sección anterior vimos cómo podemos hacer una selección de índices de frame cuando calculamos las distancias entre los elementos de dos modelos moleculares (objetos XYZ aquí). Podríamos ahora, por ejemplo, con lo aprendido hasta aquí calcular la distancia entre la selección `selection_1` del `item_1` en un cierto frame, por ejemplo el 0-ésimo, y la selección `selection_2` del `item_2` en otro cierto frame, por ejemplo el 2-ésimo:

In [38]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=0,
                         item_2=set_B, selection_2=1, frame_indices_2=2)

In [39]:
distances.shape

(1, 2, 1)

In [40]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 2 es {}'.format(distances[0,1,0]))

La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B en el frame 2 es 6.082762530298219 nm


O atendiendo a dos listas de índices de frames que serán enfrentadas secuencialmente: primer elemento de la primera lista frente a primer elemento de la segunda lista, segundo frente a segundo, y así consecutivamente:

In [41]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,1],
                         item_2=set_B, selection_2=1, frame_indices_2=[1,2])

In [42]:
distances.shape

(2, 2, 1)

In [43]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 1 es {}'.format(distances[0,1,0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 2 es {}'.format(distances[1,1,0]))

La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B en el frame 1 es 5.196152422706632 nm
La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B en el frame 2 es 6.164414002968976 nm


O el mismo cálculo con su salida en forma de diccionario:

In [44]:
distances = m3t.distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[0,1],
                         item_2=set_B, selection_2=1, frame_indices_2=[1,2], output_form='dict')

Donde dado que la relación entre índices de frame es uno a uno (el 0-ésimo de `set_A` frente al 1-ésimo de `set_B` y el 1-ésimo de `set_A` frente al 2-ésimo de `set_B`, en este caso), el diccionario resultante apela únicamente al índice original de `frame_indices_1`:

In [45]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 1 es {}'.format(distances[2][1][0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 2 es {}'.format(distances[2][1][1]))

La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B en el frame 1 es 5.196152422706632 nm
La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B en el frame 2 es 6.164414002968976 nm


De esta manera podríamos incluso enfrentar ciertos elementos del set_A en todos sus frames contra ciertos elementos de set_B de un mismo frame, por ejemplo frente al 0-ésimo.

In [46]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], frame_indices_1=[0,1,2],
                         item_2=set_B, selection_2=1, frame_indices_2=[0,0,0])

In [47]:
distances.shape

(3, 3, 1)

In [48]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[0,2,0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[1,2,0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 2 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2,2,0]))

La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B en el frame 0 es 5.385164807134504 nm
La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B en el frame 0 es 5.0990195135927845 nm
La distancia entre el elemento 2-ésimo del set_A en el frame 2 y el elemento 1-ésimo del set_B en el frame 0 es 4.242640687119285 nm


O igualmente:

In [49]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], frame_indices_1=[0,1,2],
                         item_2=set_B, selection_2=1, frame_indices_2=[0,0,0], output_form='dict')

In [50]:
print('La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2][1][0]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2][1][1]))
print('La distancia entre el elemento 2-ésimo del set_A en el frame 2 y el elemento 1-ésimo del set_B\
 en el frame 0 es {}'.format(distances[2][1][2]))

La distancia entre el elemento 2-ésimo del set_A en el frame 0 y el elemento 1-ésimo del set_B en el frame 0 es 5.385164807134504 nm
La distancia entre el elemento 2-ésimo del set_A en el frame 1 y el elemento 1-ésimo del set_B en el frame 0 es 5.0990195135927845 nm
La distancia entre el elemento 2-ésimo del set_A en el frame 2 y el elemento 1-ésimo del set_B en el frame 0 es 4.242640687119285 nm


Ya podemos entonces hacer el ejercicio de calcular distancias de desplazamiento de unos mismos elementos entre frames. Por ejemplo, veamos como podríamos responder la pregunta: ¿Cuanto se desplazan los elementos de `set_A` entre frames consecutivos?

In [51]:
distances = m3t.distance(item_1=set_A, frame_indices_1=[0,1], frame_indices_2=[1,2])

Podemos ahora ver cuanto se desplazó por ejemplo el elemento 1-ésimo de `set_A` entre los frames 0-ésimo y 1-ésimo, o entre el 1-ésimo y el 2-esimo.

In [52]:
print('La distancia de desplazamiento del elemento 1-ésimo de set_A entre los frames 0 y 1 es {}'.format(distances[0][1][1]))
print('La distancia de desplazamiento del elemento 1-ésimo de set_A entre los frames 1 y 2 es {}'.format(distances[1][1][1]))

La distancia de desplazamiento del elemento 1-ésimo de set_A entre los frames 0 y 1 es 1.0 nm
La distancia de desplazamiento del elemento 1-ésimo de set_A entre los frames 1 y 2 es 1.0 nm


El inconveniente que puede tener esta aproximación al cálculo de distancias de desplazamientos es que es absurdamente cara computacionalmente ya que también hemos tenido que calcular la distancia entre elementos distintos en frames distintos. O dicho de otra manera, para calcular la distancia de 3 elementos entre dos frames hemos tenido que calcular 9 distancias:

In [53]:
distances.shape

(2, 3, 3)

No tienen en este último caso mucho sentido los elementos fuera de la diagonal de la matriz `distances[0,:,:]`. ¿Qué significa la distancia entre la posición del elemento 0-ésimo en el frame 0-ésimo y la posición del elemento 2-ésimo en el frame 1-ésimo, por ejemplo?

In [54]:
distances[0,0,2] # En principio no tienen mucho sentido físicamente estos valores cruzados

Quantity(value=3.0, unit=nanometer)

Veamos en la siguiente sección como podríamos haber calculado el desplazamiento de los distintos elementos de `set_A` entre distintos frames sin necesidad de calcular los cruces entre distintos elementos del mismo objeto.

## Distancias entre pares

Cuando invocamos el método `molmodmt.distance` para calcular las distancias entre un conjunto de N elementos de `item_1` y M elementos de `item_2` el resultado es un objeto, tensor o diccionario, con NxM distancias por frame. Pero en ocasiones no necesitamos todas estas distancias sino que sólo queremos ciertas distancias entre pares dados por la lectura secuencial de los elementos de `selection_1` y `selection_2` uno a uno. Este cálculo se puede realizar introduciendo el argumento `pairs=True` en la llamada al método. Veamos esto mejor con un ejemplo recuperando los modelos `set_A` y `set_B`:

Calculemos en primer lugar algo sencillo para ilustrar este uso de `molmodmt.distance`. Calculemos la distancia entre los elementos 0-ésimo y 1-ésimo, y entre los elementos 0-ésimo y 2-ésimo de `set_A` para los frames 1-ésimo y 2-ésimo. 

In [55]:
distances = m3t.distance(item_1=set_A, selection_1=[0,0,1], selection_2=[0,2,2], frame_indices_1=[1,2], pairs=True)

La primera diferencia que podemos observar en el resultado frente al uso anterior de `molmodmt.distance` es el rango y dimensiones del tensor `distances`.

In [56]:
distances.shape

(2, 3)

El tensor es de rango 2, y no de rango 3 como anteriormente. Lo cual tiene mucho sentido porque efectivamente solo quisimos calcular 6 distancias correspondientes a tres pares de elementos para dos frames. Los pares son los construidos con el primer elemento de `selection_1` y el primer elemento de `selection_2`, el segundo de `selection_1` y el segundo de `selection_2`, y así sucesivamente. Es por eso que cuando `pairs=True` ambas selecciones deben tener el mísmo número de elementos. Una manera rápida de calcular los pares, si necesitas almacenarlos en una lista, es recurrir al iterador de python `zip`.

In [57]:
list_1 = [0,0,1]
list_2 = [0,2,2]
pairs = []

for ii,jj in zip(list_1,list_2):
    pairs.append([ii,jj])

print(pairs)

[[0, 0], [0, 2], [1, 2]]


Así, si necesitamos acudir al tensor resultante `distances` para ver la distancia del tercer par, en este caso [1,2], en el frame 1-ésimo:

In [58]:
print(distances[0,2])

1.4142135623730951 nm


De nuevo, el acceso a los datos resultantes puede ser un poco más cómodo si solicitamos que la salida tenga forma de diccionario de diccionarios. En este caso los indices originales de los elementos y los frames se respetan:

In [59]:
distances = m3t.distance(item_1=set_A, selection_1=[0,0,1], selection_2=[0,2,2], frame_indices_1=[1,2],
                         pairs=True, output_form='dict')

Veamos ahora la misma distancia entre el elemento 1-ésimo y el 2-ésimo para el frame 1-ésimo:

In [60]:
print(distances[1][2][1])

1.4142135623730951 nm


Esta opción `pairs=True` nos permite entonces calcular las distancias de desplazamiento de los elementos de `set_A` entre frames consecutivos de una manera más económica computacionalmente (a diferencia de lo observado en el final de la sección anterior):

In [61]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], selection_2=[0,1,2],
                         frame_indices_1=[0,1], frame_indices_2=[1,2], pairs=True)

Sólo hemos tenido que calcular los 6 desplazamientos de interes, y no los 18 que fueron necesarios cuando `pairs=False` (opción por defecto). Veamos por ejemplo el desplazamiento del elemento 2-ésimo entre el frame 1-ésimo y el 2-ésimo:

In [62]:
print(distances[1,2])

1.4142135623730951 nm


De nuevo con la opción `output_form` el acceso a los desplazamientos se hace con los índices originales:

In [63]:
distances = m3t.distance(item_1=set_A, selection_1=[0,1,2], selection_2=[0,1,2],
                         frame_indices_1=[0,1], frame_indices_2=[1,2],
                         pairs=True, output_form='dict')

Así entonces, el mismo desplazamiento del 2-ésimo elemento entre el frame 1-ésimo y el 2-ésimo:

In [64]:
print(distances[2][2][1])

1.4142135623730951 nm


## Distancias máximas y mínimas

En múchas ocasiones lo que necesitamos calcular los distancias mínimas o distancias máximas. Para facilitar la obtención de estos observables encontramos en MolModMT dos métodos específicos cuya manera de ser usados es similar: `molmodmt.minimum_distance` y `molmodmt.maximum_distance`. Trabajemos en este sección con el cálculo de distancias mínimas para ilustrar el funcionamiento de estos métodos.

Calculemos en primer lugar la distancia mínima entre los elementos de `set_A` y los de `set_B` para cada uno de los tres frames:

In [65]:
min_pairs, min_distances = m3t.minimum_distance(item_1=set_A, item_2=set_B)

El resultado son dos objetos. El primero (aquí `min_pairs`) recoje la pareja de elementos, uno del item_1 y el otro del item_2 cuya distancia es mínima entre `set_A` y `set_B`, para cada frame. Y el segundo es el vector de distancias mínimas para cada frame. Por ejemplo veamos cúal es la distancia mínima entre `set_A` y `set_B` y cuales el par de elementos que más próximos están para cada frame:

In [66]:
for ii in range(3):
    print('Frame {}: distancia mínima encontrada entre {} de set_A y {} de set_B a {}'.format(ii,min_pairs[ii,0], min_pairs[ii,1], min_distances[ii]))

Frame 0: distancia mínima encontrada entre 0 de set_A y 1 de set_B a 3.605551275463989 nm
Frame 1: distancia mínima encontrada entre 0 de set_A y 1 de set_B a 2.449489742783178 nm
Frame 2: distancia mínima encontrada entre 1 de set_A y 1 de set_B a 4.123105625617661 nm


El orden de los elementos en los objetos de salida atienden, al igual que en la salida de `molmodmt.distance`, a su índice en `selection_1`, `selection_2` y `frame_indices_1`. En el ejemplo anterior los índices originales coinciden con los de las selecciones porque el valor de las tres es `'all'`. Pero esto no tiene por que ser siempre así. Veamos otro ejemplo:

In [67]:
min_pairs, min_distances = m3t.minimum_distance(item_1=set_A, selection_1=[1,2], item_2=set_B, selection_2=[0,1],
                                               frame_indices_1=[1,2])

El primer par que minimiza la distancia para el primer frame seleccionado es:

In [68]:
print(min_pairs[0])

[0 1]


Y no se refiere a los elemento 0-ésimo de `set_A` frente al 1-ésimos de `set_B` para el frame 0-ésimo, sino al 0-ésimo elemento de `selection_1` frente al 1-ésimo de `selection_2` para el frame 0-ésimo de la lista `frame_indices_1`. Es decir, para el frame 1-ésimo el elemento 1-ésimo de `set_A` y el 1-ésimo de `set_B` minimizan la distancia de todos los pares posibles entre los elementos de cada selección de los modelos. Y la distancia entre estos es:

In [69]:
print(min_distances[0])

4.242640687119285 nm


Una variante a este primer uso de `molmodmt.minimum_distance` es el que responde a la siguiente situación. En el primer caso anterior hemos buscado la pareja de elementos que minimiza la distancia entre los dos modelos moleculares. ¿Cómo ahora puedo calcular las distancias mínimas de cada uno de los elementos de un modelo con todos los del otro elemento? O dicho de otra manera, cual es el elemento de `set_B` más próximo al 0-ésimo de `set_A`, ¿y al 1-ésimo de `set_B`? Necesitamos entonces no considerar `set_A` como una entidad compuesta por elementos, si no por una lista de elementos que van a ser considerados individualmente en la busqueda del criterio extremal de la distancia. Para esto tenemos los argumentos `as_entity_1=False` o `as_entity_2=False` según convenga.  

In [70]:
min_pairs, min_distances = m3t.minimum_distance(item_1=set_A, selection_1=[1,2], frame_indices_1=[1,2],
                                                item_2=set_B, selection_2=[0,1], frame_indices_2=[1,2],
                                                as_entity_1=False, as_entity_2=True)

Ahora el rango y dimensión de los dos objetos de salida se ajusta a los criterios de entrada solicitados. `item_1` se ha considerado como un conjunto de elementos independientes dados por `selection_1`, e `item_2` como un todo compuesto por los elementos seleccionados por `selection_2`.

In [71]:
print(min_pairs.shape, min_distances.shape)

(2, 2) (2, 2)


Para ambos objetos el primer rango corresponde al índice de frame de la lista `frame_indices_1`. Y el segundo rango da cuenta de los elementos de `set_A` según `selection_1`. Así el elemento de `set_B` más próximo al elemento 2-ésimo de `set_A` es para el frame 1-ésimo es:

In [72]:
min_pairs[0,1]

1

El elemento 1-ésimo de la selección `selection_2`, osea el elemento 0-ésimo de `set_B`, es el que está más próximo al elemento 2-ésimo de `set_A` para el frame 1-ésimo, y su distancia es:

In [73]:
print(min_distances[0,1])

5.0990195135927845 nm


Por último, con estos métodos podemos, por ejemplo, saber qué elemento de un modelo fue el que más o el que menos se desplazo entre frames consecutivos:

In [74]:
max_pairs, max_distances = m3t.maximum_distance(item_1=set_A, selection_1=[0,1,2], selection_2=[0,1,2],
                                                frame_indices_1=[0,1], frame_indices_2=[1,2], pairs=True)

El elemento que más se desplaza de `set_A` entre el frame 1-ésimo y el 2-ésimo es:

In [75]:
max_pairs[1]

2

Es el elemento 2-ésimo de `set_A`. Es decir, la pareja formada por el elemento 2-ésimo de `selection_1` y el 2-ésimo de `selection_2` que coincide que es la pareja de los elementos 2-ésimos de `set_A`. ¿Por qué una pareja? porque se trata de la distancia entre el elemento 2-ésimo de `set_A` en el frame 1-ésimo, según frame_indices_1, y el elemento 2-ésimo de `set_B` en el frame 2-ésimo, según frame_indices_2. Y la magnitud del desplazamiento de este elemento es para este frame es:

In [76]:
print(max_distances[1])

1.4142135623730951 nm


## Listas de vecinos

En esta sección veremos cómo usar el método `molmodmt.neighbors_lists` para calcular listas de vecinos. Estas listas de vecinos pueden ser definidas de dos maneras según tomen valor uno de los dos argumentos de entrada `num_neighbors` o `threshold`.

## Mapas de contactos

## Distancias entre grupos 

## Distancias mínimas o máximas entre grupos

## Listas de vecinos entre grupos

## Mapas de contactos entre grupos

In [77]:
molsys = m3t.load('/home/diego/1brs.pdb', selection='chainid 0 or chainid 3')

In [78]:
m3t.view(molsys)

NGLWidget()

In [79]:
dists_CAs = m3t.distance(molsys, selection_1="chainid 0 and name CA", selection_2="chainid 3 and name CA")

In [80]:
chain0_CAs = m3t.select(molsys, "chainid 0 and name CA")
chain3_CAs = m3t.select(molsys, "chainid 3 and name CA")

ii=10
jj=20
frame_index=0

print('La distancia entre el átomo {} y el átomo {} es {}'.format(chain0_CAs[ii], chain3_CAs[jj], dists_CAs[frame_index,ii,jj]))

La distancia entre el átomo 74 y el átomo 1022 es 3.475024595596411 nm


In [81]:
dists_CAs = m3t.distance(molsys, selection_1="chainid 0 and name CA", selection_2="chainid 3 and name CA", output_form='dict')

In [82]:
ii=74
jj=1022
frame_index=0

print('La distancia entre el átomo {} y el átomo {} es {}'.format(ii, jj, dists_CAs[ii][jj][frame_index]))

La distancia entre el átomo 74 y el átomo 1022 es 3.475024595596411 nm


In [83]:
m3t.get(molsys, target='atom', indices=[74,1022], index=True, id=True, name=True, element=True)

[[74, 1022], [75, 2745], ['CA', 'CA'], ['C', 'C']]